# StepFun 多模态推理与 Reasoning 获取示例

本 Notebook 演示了如何使用 StepFun 平台提供的 `OpenAI` 兼容接口，调用多模态模型（如 `step-3`）进行图片分析，并展示如何获取模型在流式输出中的**推理过程 (`reasoning`)**。

参考文档：[https://platform.stepfun.com/docs/guide/reasoning](https://platform.stepfun.com/docs/guide/reasoning#%E8%8E%B7%E5%8F%96-reasoning-%E5%86%85%E5%AE%B9)

### 前置条件

1.  **安装依赖：** `pip install openai`
2.  **设置环境变量：** 确保您的运行环境中已设置 `STEPFUN_API_KEY` 和 `STEPFUN_ENDPOINT`。
3.  **图片文件：** 请在 Notebook 所在目录创建 `./media/` 文件夹，并放入一个名为 `test-监测.png` 的图片文件用于测试。


In [ ]:
from openai import OpenAI
import time
import base64
import os
import sys

# 检查环境变量是否设置
try:
    STEP_API_KEY = os.environ["STEPFUN_API_KEY"]
    BASE_URL = os.environ['STEPFUN_ENDPOINT']
except KeyError:
    print("❌ 错误：请设置 STEPFUN_API_KEY 和 STEPFUN_ENDPOINT 环境变量。")
    sys.exit(1)

# --- 配置 --- 

# 初始化 OpenAI 客户端
client = OpenAI(api_key=STEP_API_KEY, base_url=BASE_URL)

# 必填-选择模型 (注意：多模态和 Reasoning 功能通常在步长较大的模型中支持)
COMPLETION_MODEL = "step-3"
#COMPLETION_MODEL = "step-1-32k"
#COMPLETION_MODEL = "step-1-256k"
#COMPLETION_MODEL = "step-1v-mini"
#COMPLETION_MODEL = "step-2-16k"
#COMPLETION_MODEL = "step-2-mini"

# --- 辅助函数：图片转 base64 ---

def image_to_base64(image_path):
    """将本地图片文件转换为 base64 字符串"""
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"未找到图片文件: {image_path}")
        
    with open(image_path, "rb") as image_file:
        # 检查文件是否为空
        file_content = image_file.read()
        if not file_content:
            raise ValueError(f"图片文件为空: {image_path}")
            
        encoded_string = base64.b64encode(file_content)
    return encoded_string.decode('utf-8')

# --- 准备图片和消息内容 ---

# 注意提供准确的图片路径
image_path = "./media/01_鱼香肉丝.jpg"


try:
    bstring = image_to_base64(image_path)
except (FileNotFoundError, ValueError) as e:
    print(f"🚫 文件准备失败: {e}")
    print("请确保图片路径存在。")
    sys.exit(1)

sys_prompt = """你是由阶跃星辰提供的AI聊天助手，你擅长中文，英文，以及多种其他语言的对话。在保证用户数据安全的前提下，你能对用户的问题和请求，作出快速和精准的回答。同时，你的回答和建议应该拒绝黄赌毒，暴力恐怖主义的内容。"""
user_prompt = "帮我看看这是什么菜，如何制作？"

# content 为包含图片和文本的 multipart 消息列表
messages = [
    {"role": "system", "content": sys_prompt}, 
    {"role": "user", "content": [
        {"type": "image_url", "image_url": {"url": f"data:image/jpg;base64,{bstring}", "detail": "high"}}, # 注意：这里将 jpg和 png要匹配文件名
        {"type": "text", "text": user_prompt}
    ]}
]

print(f"模型: {COMPLETION_MODEL}")
print(f"用户提示: {user_prompt}")


模型: step-3
用户提示: 帮我看看这是什么菜，如何制作？


## 执行推理并获取 Reasoning

以下代码将以流式（`stream=True`）方式调用模型，并特别检查 `chunk.choices[0].delta` 中是否包含 `reasoning` 字段，从而将其打印出来。

注意：`reasoning` 字段通常在流的开始部分返回。

In [4]:
time_start = time.time()
print("\n--- 开始流式生成 ---")

# 实现文本补全
response = client.chat.completions.create(
    messages=messages,
    model=COMPLETION_MODEL,
    stream=True,
)

i = 0
for chunk in response:
    # 检查 delta 对象中是否有 reasoning 属性
    if hasattr(chunk.choices[0].delta, 'reasoning') and chunk.choices[0].delta.reasoning:
        reasoning = chunk.choices[0].delta.reasoning
        print("\n\n[模型思考过程 Reasoning]:", reasoning)
        
    # 打印正常的文本内容
    content = chunk.choices[0].delta.content
    if content:
        print(content.strip(), end="", flush=True)
        
    if i == 0:  # 记录首字时间
        time_firstWord = time.time()
        elapsed_time = time_firstWord - time_start
        print(f"\n\n首字生成时间: {elapsed_time:.2f} 秒")
    i += 1

time_end = time.time()
elapsed_time = time_end - time_start
print("\n\n--- 生成结束 ---")
print("当前模型为:", COMPLETION_MODEL)
print(f"总生成时间: {elapsed_time:.2f} 秒")


--- 开始流式生成 ---


首字生成时间: 1.92 秒


[模型思考过程 Reasoning]: 这张


[模型思考过程 Reasoning]: 图片


[模型思考过程 Reasoning]: 展示


[模型思考过程 Reasoning]: 的


[模型思考过程 Reasoning]: 菜品


[模型思考过程 Reasoning]: 是


[模型思考过程 Reasoning]: **


[模型思考过程 Reasoning]: 鱼


[模型思考过程 Reasoning]: 香


[模型思考过程 Reasoning]: 肉


[模型思考过程 Reasoning]: 丝


[模型思考过程 Reasoning]: **


[模型思考过程 Reasoning]: ，


[模型思考过程 Reasoning]: 它是


[模型思考过程 Reasoning]: 中国


[模型思考过程 Reasoning]: 经典的


[模型思考过程 Reasoning]: 川


[模型思考过程 Reasoning]: 菜


[模型思考过程 Reasoning]: 之一


[模型思考过程 Reasoning]: ，


[模型思考过程 Reasoning]: 以


[模型思考过程 Reasoning]: “


[模型思考过程 Reasoning]: 鱼


[模型思考过程 Reasoning]: 香


[模型思考过程 Reasoning]: ”


[模型思考过程 Reasoning]: 风味


[模型思考过程 Reasoning]: （


[模型思考过程 Reasoning]: 咸


[模型思考过程 Reasoning]: 、


[模型思考过程 Reasoning]: 甜


[模型思考过程 Reasoning]: 、


[模型思考过程 Reasoning]: 酸


[模型思考过程 Reasoning]: 、


[模型思考过程 Reasoning]: 辣


[模型思考过程 Reasoning]: 、


[模型思考过程 Reasoning]: 鲜


[模型思考过程 Reasoning]: 兼具


[模型思考过程 Reasoning]: ）


[模型思考过程 Reasoning]: 为


[模型思考过程 Reasonin